In [24]:
import pandas as pd
import math
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
import statsmodels.api as sm
from scipy.stats import norm
from statsmodels.sandbox.regression.gmm import GMM

In [25]:
def comp_outside_good(data,name):
    #pre-processing to calculate outside good shares
    shares = data[['Market_ID',name]].copy()

    group_shares = shares.groupby('Market_ID').sum()
    group_shares['Outside Good Share'] = 1 - group_shares[name]

    data = pd.merge(data,group_shares[['Outside Good Share']], 
                right_index=True, left_on = 'Market_ID')
    return data

data = pd.read_csv('data.csv')
data = comp_outside_good(data,'Inside Good Share')

In [48]:
def setup_data(data):
    """simplify setting up data correctly"""
    #calculate hausmann insturments
    mkt_dum = pd.get_dummies(data['Market_ID'],prefix='mkt',drop_first=True)
    plan_dum = pd.get_dummies(data['Plan_ID'],prefix='plan',drop_first=True)
    exog = np.array( data[['Network Score','Satisfaction Score', 'PPO']])
    
    #concat hausman instr with exog variables
    #hausman_instr = np.concatenate( (exog, np.array(plan_dum)), axis =1 )
    
    hausman_instr = plan_dum
    
    #set up x and y
    y = data[['Inside Good Share','Outside Good Share']]
    x =  data[['Network Score','Satisfaction Score','PPO','Premium']]
    return x,y,hausman_instr

x,y,z =  setup_data(data)

In [27]:
#set up useful global variables 
NMKTS = data['Market_ID'].nunique()
NPLANS = data['Plan_ID'].nunique()
NOBS = data['Plan_ID'].count()
NSIM = 60

theta1 = np.array([3.5,2,1,-2])
theta2 = np.array([.15,.1,.5]) # initialize theta2 for testing purposes
delta = np.ones(NOBS)*(-2)

#print global variables
print NMKTS,NPLANS,NOBS

600 16 3300


In [28]:
#set up random draws v
def gen_newsim():
    mkt_ids = np.array(data['Market_ID'])
    v = [0]*NOBS
    for mkt in range(1,601):
        v_i = np.random.normal(size=(3,NSIM))
        for i in range(NOBS):
            if mkt_ids[i] == mkt:
                v[i] = v_i
    return np.array(v).transpose()
        
v = gen_newsim()
#np.savetxt("simulations.csv", v.reshape(3*NSIM,3300), delimiter=",")

In [29]:
#use same simulations each time
v = np.genfromtxt('simulations.csv', delimiter=',').reshape(NSIM,3,3300)
print v.shape

(60, 3, 3300)


# Part 1 - Estimation

## Calculating $\delta_{jt}$, $\xi_{jt}$

In [30]:
def cal_sim_s(data, v, delta, theta2):
    """calculate market share for each simulated consumer"""
    
    #copy x and delta for simulations using tiling
    x =  np.array(data.copy()[['Network Score','Satisfaction Score','PPO']]).transpose()
    x  = np.tile(x,(NSIM,1,1))
    theta2 = np.tile( np.array([theta2]).transpose()  ,(NSIM,1,3300))
    delta  = np.tile( delta  ,(NSIM,1))
    
    #add to calcualte market shares
    sim_exp = pd.DataFrame( np.exp(delta + (theta2*v*x).sum(axis=1)).transpose() ) 
    
    #sum up between markets
    sim_exp['mkt_id'] = data['Market_ID']
    sum_exp = sim_exp.groupby('mkt_id').sum() 
    sum_exp = pd.merge(data.copy()[['Market_ID']], sum_exp, 
                       right_index=True, left_on = 'Market_ID')
    
    #format so I can broadcast
    sim_exp = np.array(sim_exp).transpose()[:-1]
    sum_exp = np.array(sum_exp).transpose()[1:] + 1
    
    return sim_exp/sum_exp




print s.mean(), s.min(), s.max(), s.shape

0.07715041512097259 0.060073812123759084 0.12115866369684081 (3300,)


In [31]:
def cal_delta(data, v, theta2, error = 1e-3, maxiter = 500):
    """Calculate mean utility via contraction mapping"""

    niter = 0
    
    #initialize loop parameters
    delta = np.zeros(NOBS)
    s = cal_s(data, v, delta, theta2)
    diff = np.log(data['Inside Good Share']) - np.log(s)
    
    
    while ((abs(diff).max() > 1e-6) #this is easier to converge
           and (abs(diff).mean() > error) 
           and niter < maxiter):
        
        s = cal_s(data, v, delta, theta2)
        diff = np.log(data['Inside Good Share']) - np.log(s)

        if np.isnan(diff).sum():
            raise Exception('nan in diffs')
            
        delta += diff
        niter += 1

    return delta


delta = cal_delta(data, v, theta2)
print delta.shape
print delta.mean()

(3300,)
0.105590877211


In [32]:
#initialize theta1 i.e. alpha and beta
def cal_xi(data, delta, theta1):
    """Calculate xi with F.O.C"""
    x,y,z =  setup_data(data)
    xi = delta - np.matmul(np.array(x),theta1)
    return  xi


xi = cal_xi(data, delta,  theta1)
print xi.max(), xi.min(), xi.mean()

0.3957873067869968 -0.6655375880879966 -0.191204911395


## Calculating  $\theta_1,\theta_2$

where $\theta_1 = (\alpha, \beta)$

I only solve GMM over $\theta_2$, the non-linear parameters

In [52]:
def cal_theta1(data, delta, T):
    """  calculate theta 1 using FOCs (X1'Z T Z'X )^-1 X1'Z T Z' delta """
    #set up variables
    x,y,z =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    
    #build up to main equation
    XtZ = X.transpose().dot(Z)
    ZtX = Z.transpose().dot(X)
        
    first_exp = np.linalg.inv( XtZ.dot(T).dot(ZtX))
    second_exp = XtZ.dot(T).dot(Z.transpose()).dot(delta)
    theta1 = first_exp.dot(second_exp)
    return theta1



T =  np.linalg.inv( z.transpose().dot(z) )
print cal_theta1(data, delta, T)

[ 3.45230392  2.08076381  0.82731562 -2.05340375]


In [53]:
def gmm_objective(theta2_init, data, v,  T):
    """Z'ww'Z of theta2"""
    #set up variables
    x,y,z =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    
    #do calculations
    delta = cal_delta(data, v, theta2_init)
    theta1 = cal_theta1(data, delta, T)
    xi = cal_xi(data, delta, theta1)
    return xi.dot(Z).dot(T).dot(Z.transpose()).dot(xi)
    

def calc_theta2(data, v, T, theta2_init):
    """calculate theta2 using scipy"""
    #theta2 = minimize(gmm_objective, theta2_init, args=(data, v,  T), method='Nelder-Mead',
    #                  options={'maxiter':100, 'disp': True, 'xatol':1e-3, 'fatol':5})
    theta2 = minimize(gmm_objective, theta2_init, args=(data, v,  T), method='BFGS',
                      options={'maxiter':100, 'disp': True})
    return abs(theta2.x)



#run some tests
theta2_init = np.array([.25,.05,1])
theta2_test = calc_theta2( data, v, T, theta2_init)

print theta2_test

Optimization terminated successfully.
         Current function value: 0.396805
         Iterations: 8
         Function evaluations: 45
         Gradient evaluations: 9
[0.21249848 0.07349582 1.16048071]


In [54]:
theta2_init = np.array([.25,.05,1])

def calc_theta(data, v, theta2_init):
    
    #initialize theta
    x,y,z =  setup_data(data)
    X,Z = np.array(x), np.array(z)
    theta2 = theta2_init 
    
    #on first step, use consistent approximation of T
    T =  np.linalg.inv( z.transpose().dot(z) )
    for i in range(2):  
       
        #on second use T using estimated xi
        if i==1:
            xi = cal_xi(data, delta, theta1)
            xi =np.array([xi]).transpose()
            T =  np.linalg.inv( Z.transpose().dot(xi).dot(xi.transpose()).dot(Z) )
        
        theta2 = calc_theta2(data, v, T, theta2)
        delta = cal_delta(data, v, theta2)
        theta1 = cal_theta1(data, delta, T)
        
    return theta1, theta2


theta = calc_theta(data, v, theta2_init)

print theta

Optimization terminated successfully.
         Current function value: 0.396805
         Iterations: 8
         Function evaluations: 45
         Gradient evaluations: 9
         Current function value: -0.932792
         Iterations: 0
         Function evaluations: 332
         Gradient evaluations: 64
(array([ 3.55413895,  2.09634795,  0.88263817, -2.09472103]), array([0.21249848, 0.07349582, 1.16048071]))


## Calculate Standard Errors

# Part 2 -Elasticities

In [55]:
theta1 = np.array([3.55413895,  2.09634795,  0.88263817, -2.09472103])
theta2 = np.array([0.21249848, 0.07349582, 1.16048071])

In [70]:
a = np.array([1,2,3])
b = np.array([4,5,6])
c  = np.array([a,b])
c1 = c.reshape(2,3,1)
c2 = c.reshape(2,1,3)

print np.tile(c1,(1,1,3)).shape
print np.tile(c2,(1,3,1)).shape

(2, 3, 3)
(2, 3, 3)
